In [1]:
# unused here:
# !pip install --upgrade huggingface_hub -q
# !pip install --upgrade pip
# !pip install ipywidgets

In [2]:
# BERTopic only work's properly with torch 2.2.1, higher versions you can have issues
!pip install torch==2.2.1
!pip install bertopic
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy] 
# this one is having an issue:
# !pip install bertopic[use]
 
from IPython.display import clear_output
clear_output()

In case bertopic complains to install in windows due to long paths, enable long paths to install **bertopic** in windows:
1. Open PowerShell as Administrator.
2. In the PowerShell window that opened with administrative privileges, type the following command and press Enter:
```bash
New-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name "LongPathsEnabled" -Value 1 -PropertyType DWORD -Force
```
3. Restart your computer.

In [16]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import json
import random

from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gutog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# 37s to load's the data
# 1m34s to append the data into a list
# Open the jsonl file

BabyProductsDict = []

with open('data/Baby_Products.jsonl', 'r') as file:
    # Read each line in the file
    for line in file:
        # Parse the JSON data on the line
        BabyProductsDict.append(json.loads(line))
        # Now data is a dictionary representing the JSON object
        # print(data)

In [12]:
print('len():',len(BabyProductsDict),'type:',type(BabyProductsDict))

len(): 6028884 type: <class 'list'>


In [14]:
# sample
BabyProductsDict[0]['text']

'I bought two of these for my kids for nap time at their preschool. But they also use at home to &#34;camp&#34;...lol. They seem to work pretty well are cute designs and have held up through multiple washings on the gentle cycle and air fluff or lay flat to dry. They still look brand new to me! They are a little stiff at first ( and sometimes after washing) but with use they do become softer and the inside is plenty soft for sleeping- my kiddos have not complaints!'

In [17]:
max_samples = 1000000

if len(BabyProductsDict) > max_samples:
    sampled_list = random.sample(BabyProductsDict, max_samples)
else:
    sampled_list = BabyProductsDict

# Print or use the sampled list as needed
print(len(sampled_list))  # Should be max_samples if the original list has more than max_samples items

1000000


In [19]:
sampled_list[0]

{'rating': 5.0,
 'title': 'So much easier!',
 'text': 'Makes pumping so much easier and the time goes by faster being able to do other things with your hands. Been able to feed my baby at the same time with a bottle, use my phone and even nap!',
 'images': [],
 'asin': 'B07875D5G1',
 'parent_asin': 'B07875D5G1',
 'user_id': 'AHPI24Q2J73QFGBSMU6VJMEFUPQQ',
 'timestamp': 1524526106601,
 'helpful_vote': 0,
 'verified_purchase': True}

In [3]:
# sample
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

# model
topic_model = BERTopic()